Machine Learning model for recognizing digits from the MNIST dataset

In [ ]:
!pip install torch matplotlib torchvision

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available! Device: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Check runtime settings.")

In [ ]:
# Define transformations for the training and test sets
transform = transforms.Compose(
    [transforms.ToTensor(), # converts a PIL image to a tensor and transforms the values {0,...255} --> [0, 1]
     transforms.Normalize(0.5, 0.5)]) # first tuple -> means, second tuple -> std's. output=(input-mean)/std
# The image has three color channels (RGB) and therefore the tuples have three elements, one for each color

# Batch sizez
train_batch_size = 8
test_batch_size = 4

# Download and load the training data
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batch_size,
                                          shuffle=True, num_workers=2)

# Download and load the test data
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size,
                                         shuffle=False, num_workers=2)

# Define class names
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')


# Assuming 'net' is your model and 'inputs', 'labels' are your data
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Functions to show an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize - [-1,1] --> [0,1]
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)
# images, labels = images.to(device), labels.to(device)

# Show images
imshow(torchvision.utils.make_grid(images))
# Print labels
print(' '.join(f'{classes[labels[j]]}' for j in range(train_batch_size)))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x

net = Net()

# Assuming 'net' is your model and 'inputs', 'labels' are your data
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move model to the device (GPU or CPU)
net.to(device)

# Example of moving tensors to GPU
# inputs, labels = inputs.to(device), labels.to(device)

In [ ]:
import torch.nn.functional as F

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(5):  # example with 5 epochs

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
correct = 0
total = 0

# We do not need to track gradients for evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)

        # Forward pass
        outputs = net(images)

        # Get the predicted class with the highest score
        _, predicted = torch.max(outputs.data, 1)

        # Update total and correct counts
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print accuracy
print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')


In [ ]:
images, labels = next(dataiter)

# Move the inputs and labels to the GPU
images, labels = images.to(device), labels.to(device)

# Print images
imshow(torchvision.utils.make_grid(images.cpu()))  # Move images back to CPU for visualization
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]}' for j in range(train_batch_size)))

# Get predictions from the model
outputs = net(images)

# Get the predicted class with the highest score
_, predicted = torch.max(outputs, 1)

print('Predicted:   ', ' '.join(f'{classes[predicted[j]]}' for j in range(train_batch_size)))